# **Procesamiento de la Escala de Dolor KPPS (King's Parkinson's Disease Pain Scale)**

La KPPS es un instrumento específico diseñado para evaluar la carga y los tipos de dolor en la enfermedad de Parkinson. A diferencia de escalas genéricas, la KPPS desglosa el dolor en 7 dominios fisiopatológicos (como dolor musculoesquelético, fluctuación, nocturno, etc.), lo que permite una fenotipificación precisa del paciente.

Este módulo tiene como objetivo importar, limpiar y calcular los puntajes totales y por subdominios de la KPPS para su integración en la base de datos maestra.

# **Conexión con Google Drive**

Para acceder al archivo específico de la escala KPPS almacenado en la nube, es necesario montar nuevamente la unidad de Google Drive si estamos en una nueva sesión o cuaderno.

**Montar la unidad**
El siguiente comando establece el puente entre el entorno de ejecución de Colab y tus archivos personales.

In [ ]:
from google.colab import drive

# Monta Google Drive en el directorio /content/drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Nota de ejecución:**

Si ya has montado el Drive en esta sesión, el sistema te avisará con un mensaje: Drive already mounted at /content/drive.

Si es la primera vez, deberás autorizar el acceso mediante la ventana emergente de Google.

# **Configuración del Entorno**

Para iniciar el análisis de la escala de dolor, importamos las librerías esenciales que nos permitirán manipular los datos clínicos y visualizar las distribuciones de intensidad del dolor.

**Importación de Librerías**

Para procesar los datos de la escala KPPS, necesitamos un conjunto robusto de herramientas que nos permitan desde la manipulación básica de tablas hasta la limpieza avanzada de nombres de variables y la visualización de resultados.

Ejecutamos la carga de los paquetes estándar de ciencia de datos:

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import re

**Utilidad específica en este módulo:**

**pandas y numpy:** Serán el motor del análisis. Usaremos Pandas para estructurar los 7 dominios de la KPPS y Numpy para calcular puntajes agregados manejando correctamente los valores faltantes.

**glob y os:** Nos permitirán localizar el archivo de la base de datos dentro de las carpetas de Google Drive de manera dinámica.

**re (Regular Expressions)**: Esta librería es especialmente útil aquí si necesitamos filtrar columnas que sigan patrones específicos (ej. "KPPS_1", "KPPS_2"...) dentro de una base de datos grande con muchas variables mezcladas.

**seaborn:** La utilizaremos para generar gráficos de violín o de barras que muestren la severidad del dolor en los distintos dominios.

# **Carga e Inspección del Dataset KPPS**

Una vez importadas las librerías necesarias, el siguiente paso es cargar los datos brutos de la King's Parkinson's Disease Pain Scale (KPPS). Esta base de datos contiene las puntuaciones de los ítems individuales que procesaremos para calcular los subdominios de dolor.

**Lectura del archivo Excel**

Definimos la ruta absoluta donde se encuentra alojada la base de datos unificada del proyecto y la cargamos en un DataFrame de Pandas.

In [ ]:
# 1. Definir la ruta del archivo fuente
file_path = '/content/drive/MyDrive/LCEN/08_Bases de Datos y Herramientas/08.2_ReMePARK/08.2.6_Unified ReMePARK 2024/Remepark_KPPS.xlsx'

# 2. Cargar archivo Excel en memoria
df = pd.read_excel(file_path)

# 3. Visualizar las primeras 5 filas (Auditoría rápida)
df.head()

,Consecutivo,anio.eval,fecha.eval,accum.visits,visit.numb,reg.innn,KPSS1.F,KPSS1.S,KPSS1.TOT,KPSS.D1,...,KPSS13.F,KPSS13.TOT,KPSS.D6,KPSS14.F,KPSS14.S,KPSS14.TOT,KPSS.D7,KPSS.TOTAL,KPSS Severity,MCID
0,1,2019,2019-07-01 00:00:00,1,1,248001,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2014,2014-07-12 00:00:00,11,1,213993,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2015,2015-02-07 00:00:00,11,2,213993,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2016,2016-09-02 00:00:00,11,3,213993,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2016,2016-12-08 00:00:00,11,4,213993,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **Selección de Variables y Limpieza de Datos (KPPS)**

Antes de proceder con los cálculos, es crucial aislar las variables pertinentes y asegurar la integridad de los datos. Este bloque de código selecciona dinámicamente todas las columnas relacionadas con la escala y elimina aquellos registros incompletos que podrían sesgar el análisis.

**Filtrado y Depuración**

El siguiente script identifica las columnas de interés basándose en su nombre y descarta los pacientes que tengan datos faltantes en cualquiera de esos ítems.

In [ ]:
# 1. Selección dinámica de columnas
# Se buscan todas las columnas que contengan el patrón de texto 'KPSS'
# Nota: Verificar si el prefijo en tu Excel es 'KPSS' o 'KPPS'
kpss_columns = df.filter(like='KPSS').columns

# 2. Eliminación de datos incompletos (Listwise Deletion)
# Se eliminan las filas que tengan al menos un valor NaN en las columnas seleccionadas
df_cleaned = df.dropna(subset=kpss_columns)

# 3. Verificación de resultados
display(df_cleaned.head())

,Consecutivo,anio.eval,fecha.eval,accum.visits,visit.numb,reg.innn,KPSS1.F,KPSS1.S,KPSS1.TOT,KPSS.D1,...,KPSS13.F,KPSS13.TOT,KPSS.D6,KPSS14.F,KPSS14.S,KPSS14.TOT,KPSS.D7,KPSS.TOTAL,KPSS Severity,MCID
7,8,2019,2019-06-03 00:00:00,11,7,213993,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No pain,NaN
8,24,2019,2019-10-06 00:00:00,8,8,196860,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No pain,NaN
23,39,2020,2/27/2020,13,10,197087,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No pain,NaN
38,40,2019,6/17/2019,13,9,197087,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No pain,NaN
39,42,2021,5/27/2021,13,12,197087,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No pain,NaN


**df.filter(like='KPSS'):** Este método es muy potente porque evita tener que escribir una lista manual de columnas (ej. ['KPSS_1', 'KPSS_2', ...]). Selecciona automáticamente cualquier columna cuyo nombre contenga la cadena de texto especificada.

**Integridad del Análisis:** Al usar dropna(subset=...), estamos aplicando un criterio estricto: solo analizaremos pacientes que hayan respondido a todos los ítems de la escala. Esto es necesario para calcular puntajes totales válidos, aunque reduce el tamaño de la muestra $(N)$.

⚠️ **Advertencia de Nomenclatura:** El código busca el patrón 'KPSS'. Asegúrate de que tus columnas en el Excel no estén etiquetadas como 'KPPS' (con doble P), ya que es la abreviatura estándar. Si es así, ajusta el filtro a .filter(like='KPPS').

# **Identificación de Sujetos Únicos**

Tras la depuración de datos faltantes, es necesario verificar cuántos pacientes individuales permanecen en el estudio. Utilizamos la variable identificadora reg.innn (Registro INNN) para contar los sujetos únicos y detectar posibles duplicados o inconsistencias.

**Extracción y Conteo de Identificadores**

El siguiente código extrae la lista de identificadores únicos del DataFrame limpio e imprime tanto el listado como el recuento total.

In [ ]:
# 1. Obtener los identificadores únicos
# Se asume que 'reg.innn' es la clave primaria o ID del paciente
unique_subjects = df_cleaned['reg.innn'].unique()

# 2. Mostrar los resultados
print("Unique subjects (reg.innn):")
print(unique_subjects)

# 3. Mostrar el conteo total
print(f"\nNumber of unique subjects: {len(unique_subjects)}")

Unique subjects (reg.innn):
[ 213993  196860  197087  226413  275573  266346  175834  256264  225532
  222126  218580  199586  254059  205645  204716  214421  213348  278474
  211904  213048  212685  263070  190926  200207  201396  205385  262731
  232838  213008  231564  226192  255443  188980  238751  254953  221824
  244636  210631  212763  257391  245221  248834  208475  216922  242198
  255346  202741  272084  255301  240515  255824  182729  207804  199867
  224350  213221  223745  209183  246351  236045  238444  249764  213185
  234222  205795  203418  253728  179028  191703  256972  242480  211445
  159662  188215  203098  205560  215502  250544  260686  260841  200756
  192823  206999  133232  200006  212089  208937  246404  247243  264244
  249035  254103  236382  221531  223417  207662  164684  186457  247915
  209387  217548  213343  226328  215038  218336  253450  268844  228981
  228113  238458  195085  206589  240386  141279  256675  206547  195761
  240604  143927  25541

**Interpretación de la Salida:**

**unique():** Este método de Pandas escanea la columna especificada y devuelve un array con cada valor distinto que aparece.

**len(...):** Nos da el tamaño de la muestra final $(N)$. Este es el número que deberás reportar en la sección de "Resultados" o en el diagrama de flujo (Flowchart) del estudio.

**Nota de Calidad:** Si el número de "unique subjects" es menor al número de filas (len(df_cleaned)), significa que tienes filas duplicadas para algunos pacientes (quizás visitas de seguimiento o errores de duplicación). Si el diseño del estudio es transversal (una sola visita por paciente), deberías investigar esto.

# **Detección de Visitas Múltiples (Análisis Longitudinal)**

Para realizar análisis de evolución (por ejemplo, "cambio en el dolor a través del tiempo"), necesitamos identificar qué pacientes tienen más de una evaluación registrada en la base de datos. Este código agrupa los registros por identificador de paciente (reg.innn) y filtra aquellos que aparecen 2 o más veces.

**Filtrado de la Cohorte Longitudinal**

El siguiente script cuenta la frecuencia de aparición de cada sujeto y extrae la lista de aquellos con múltiples entradas.

In [ ]:
# 1. Contar visitas por sujeto
# 'value_counts()' devuelve una Serie donde el índice es el ID y el valor es la frecuencia
visit_counts = df_cleaned['reg.innn'].value_counts()

# 2. Filtrar sujetos con 2 o más visitas
# Seleccionamos solo aquellos índices (IDs) cuya frecuencia sea >= 2
subjects_multiple_visits = visit_counts[visit_counts >= 2].index.tolist()

# 3. Mostrar resultados
print("Unique subjects with two or more visits:")
print(subjects_multiple_visits)
print(f"\nNumber of unique subjects with two or more visits: {len(subjects_multiple_visits)}")

Unique subjects with two or more visits:
[228113, 208352, 207662, 219672, 255443, 238389, 234702, 193834, 205560, 232319, 231539, 228666, 256022, 208232, 253450, 195085, 260841, 223689, 225320, 213185, 221969, 209183, 211686, 256675, 201975, 216364, 220833, 248439, 218144, 211522, 240141, 209006, 279789, 240857, 260205, 162605, 212763, 205078, 213267, 283671, 255301, 258838, 226320, 215038, 209387, 206589, 213343, 230447, 205677, 216811, 240881, 266013, 195761, 225885, 260742, 264834, 210303, 222126, 275573, 260686, 256806, 243054, 220277, 197087, 175834, 213993, 214864, 213008, 226192, 241737, 205385, 200006, 212089, 221531, 285243, 250696, 266477, 282274, 206999, 256647, 211445, 188563, 269581, 286122, 284252, 268393, 277199, 268658, 221624, 206619, 279356, 278830, 225593, 288235, 278474, 276658, 266842, 240685, 271996, 265392, 246351, 199867, 261258, 282619, 216098, 243055, 202741, 255346, 257391, 188980, 242480, 175066, 277618, 207150, 270672, 284113, 162697, 258098, 143927, 240604

# **Integración de Bases de Datos: Carga de Fuentes Adicionales**

Para contextualizar los hallazgos de dolor (KPPS), es necesario cruzar la información con otras variables clínicas y demográficas del estudio ReMePARK. Este bloque prepara el entorno cargando tres bases de datos complementarias fundamentales.

**Estrategia de Fusión (Merge Strategy)**

El objetivo final es unir estos archivos utilizando dos llaves primarias (Primary Keys) que identifican inequívocamente cada evento clínico:

reg.innn: Identificador único del paciente.

visit.numb: Número de la visita (para distinguir Baseline de seguimientos).

**Carga de Archivos**

El siguiente código define las rutas y carga los DataFrames correspondientes a Sociodemográficos, Escala Motora (MDS-UPDRS) y Calidad de Vida (PDQ-39).

In [ ]:
# 1. Definición de rutas de archivos (File Paths)
sociodemograph_path = '/content/drive/MyDrive/LCEN/08_Bases de Datos y Herramientas/08.2_ReMePARK/08.2.6_Unified ReMePARK 2024/Remepark_sociodemograph.xlsx'
mds_updrs_path = '/content/drive/MyDrive/LCEN/08_Bases de Datos y Herramientas/08.2_ReMePARK/08.2.6_Unified ReMePARK 2024/Remepark_MDS-UPDRS.xlsx'
pdq_path = '/content/drive/MyDrive/LCEN/08_Bases de Datos y Herramientas/08.2_ReMePARK/08.2.6_Unified ReMePARK 2024/Remepark_PDQ.xlsx'

# 2. Carga de los archivos Excel en DataFrames independientes
df_sociodemograph = pd.read_excel(sociodemograph_path)
df_mds_updrs = pd.read_excel(mds_updrs_path)
df_pdq = pd.read_excel(pdq_path)

# 3. Verificación de carga (Head)
print("Head of Remepark_sociodemograph.xlsx:")
display(df_sociodemograph.head())

print("\nHead of Remepark_MDS-UPDRS.xlsx:")
display(df_mds_updrs.head())

print("\nHead of Remepark_PDQ.xlsx:")
display(df_pdq.head())

Head of Remepark_sociodemograph.xlsx:


,num.consec,fila_id,anio.eval,fecha.eval,accum.visits,visit.numb,reg.innn,nombre,sex,anio.nac,...,megliti.use,megliti.dose,sfu.use,sfu.dose,tzd.use,tzd.dose,insul.use,insul.dose,stat.use,stat.dose
0,1,ABAT_8001_19_v01,2019,2019-01-07,1,1,248001,Abatecola X Giandomenico,0,1953.0,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
1,2,FIGU_3993_14_v01,2014,2014-12-07,11,1,213993,Figueroa Cazarez Abel,0,1957.0,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
2,3,FIGU_3993_15_v02,2015,2015-07-02,11,2,213993,Figueroa Cazarez Abel,0,1957.0,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
3,4,FIGU_3993_16_v03,2016,2016-02-09,11,3,213993,Figueroa Cazarez Abel,0,1957.0,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
4,5,FIGU_3993_16_v04,2016,2016-08-12,11,4,213993,Figueroa Cazarez Abel,0,1957.0,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN



Head of Remepark_MDS-UPDRS.xlsx:


,num.consec,fila_id,anio.eval,fecha.eval,accum.visits,visit.numb,reg.innn,UPDRS1A,UPDRS1.1,UPDRS1.2,...,Interf,EHYY,UPDRS4.1,UPDRS4.2,UPDRS4.3,UPDRS4.4,UPDRS4.5,UPDRS4.6,UPDRS4.TOTAL,UPDRS.TOTAL
0,879,DAVI_1307_12_v02,2012,2012-12-04 00:00:00,2,2,41307,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,880,DAVI_1307_11_v01,2011,2011-05-07 00:00:00,2,1,41307,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,1177,FERR_0807_13_v01,2013,7/15/2013,2,1,70807,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,1178,FERR_0807_13_v02,2013,7/19/2013,2,2,70807,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,958,DIAZ_8927_11_v01,2011,1/26/2011,3,1,88927,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0



Head of Remepark_PDQ.xlsx:


,Consecutivo,anio.eval,fecha.eval,accum.visits,visit.numb,reg.innn,PDQ.1,PDQ.2,PDQ.3,PDQ.4,...,PDQ8.index,PDQ39.mov.index,PDQ39.daily.index,PDQ39.Emotion.index,PDQ39.stigma.index,PDQ39.social.index,PDQ39.cogni.index,PDQ39.comm.index,PDQ39.discom.index,PDQ39.SI.index
0,1,2019,2019-07-01 00:00:00,1,1,248001,3.0,NaN,NaN,NaN,...,31.250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2014,2014-07-12 00:00:00,11,1,213993,4.0,NaN,NaN,NaN,...,37.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2015,2015-02-07 00:00:00,11,2,213993,4.0,NaN,NaN,NaN,...,34.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2016,2016-09-02 00:00:00,11,3,213993,1.0,NaN,NaN,NaN,...,15.625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2016,2016-12-08 00:00:00,11,4,213993,1.0,NaN,NaN,NaN,...,43.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Descripción de las Fuentes de Datos:**

**df_sociodemograph: **Contiene variables estáticas o semi-estáticas como edad, género, años de escolaridad y duración de la enfermedad. Esencial para caracterizar la cohorte.

**df_mds_updrs:** Contiene la Movement Disorder Society - Unified Parkinson's Disease Rating Scale. Nos permitirá correlacionar la severidad motora con la intensidad del dolor.

**df_pdq (PDQ-39):** El Parkinson's Disease Questionnaire evalúa la calidad de vida. Nos permitirá medir el impacto del dolor en el bienestar general del paciente.

# **Fusión de Bases de Datos (Merging)**

Una vez cargadas las tablas individuales (Sociodemográficos, UPDRS, PDQ y nuestra tabla limpia de KPPS), procedemos a unificarlas en una sola estructura analítica maestra. Este proceso se realiza de forma secuencial, añadiendo una nueva dimensión de datos en cada paso.

**Ejecución de la Fusión (Outer Join)**

El siguiente código utiliza la función pd.merge para combinar los DataFrames. La unión se realiza utilizando dos llaves compuestas: el identificador del sujeto (reg.innn) y el número de visita (visit.numb).

**Reasoning**:
Merge the dataframes one by one using 'reg.innn' and 'visit.numb' as keys and an outer merge.



In [ ]:
# 1. Primera fusión: KPPS + Sociodemográficos
# Se añaden sufijos explícitos para evitar confusiones si hay columnas con nombres iguales
df_merged = pd.merge(df_cleaned, df_sociodemograph,
                     on=['reg.innn', 'visit.numb'],
                     how='outer',
                     suffixes=('_kpps', '_sociodemograph'))

# 2. Segunda fusión: Resultado anterior + MDS-UPDRS (Motor)
# Nota: Al dejar el sufijo vacío (''), mantenemos los nombres originales de la tabla izquierda
df_merged = pd.merge(df_merged, df_mds_updrs,
                     on=['reg.innn', 'visit.numb'],
                     how='outer',
                     suffixes=('', '_mds_updrs'))

# 3. Tercera fusión: Resultado anterior + PDQ-39 (Calidad de Vida)
df_merged = pd.merge(df_merged, df_pdq,
                     on=['reg.innn', 'visit.numb'],
                     how='outer',
                     suffixes=('', '_pdq'))

# 4. Verificación final
# Visualizamos la cabecera para confirmar que las columnas se han añadido horizontalmente
display(df_merged.head())

# Verificamos las dimensiones finales de la matriz (Filas, Columnas)
print(f"\nShape of the final merged DataFrame: {df_merged.shape}")

,Consecutivo,anio.eval_kpps,fecha.eval_kpps,accum.visits_kpps,visit.numb,reg.innn,KPSS1.F,KPSS1.S,KPSS1.TOT,KPSS.D1,...,PDQ8.index,PDQ39.mov.index,PDQ39.daily.index,PDQ39.Emotion.index,PDQ39.stigma.index,PDQ39.social.index,PDQ39.cogni.index,PDQ39.comm.index,PDQ39.discom.index,PDQ39.SI.index
0,NaN,NaN,NaN,NaN,1,41307,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,2,41307,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,1,70807,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2,70807,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,1,88927,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Shape of the final merged DataFrame: (4801, 311)


**Justificación Metodológica: how='outer'**

En este procedimiento hemos seleccionado una estrategia de unión tipo Outer Join (Unión Externa Completa).

**¿Qué hace?** Retiene todas las filas de ambas tablas. Si un paciente tiene datos en KPPS pero no en UPDRS (o viceversa), el paciente permanece en la base de datos y los campos faltantes se rellenan con NaN.

**¿Por qué es importante?** En estudios clínicos complejos, es común que falten algunos formularios. Usar un inner join (intersección) sería muy agresivo, ya que descartaría a cualquier sujeto que no tenga el set de datos 100% perfecto, reduciendo drásticamente el tamaño de la muestra $(N)$. El outer join maximiza la retención de datos para análisis posteriores.

# **Verificación y Auditoría de la Base de Datos Unificada**

Tras realizar la fusión de múltiples fuentes (KPPS, Sociodemográficos, MDS-UPDRS, PDQ), es imperativo auditar el objeto resultante. Este paso confirma que la integración se realizó correctamente y nos da una visión global del tamaño final de nuestra cohorte analítica.

**Inspección Visual y Dimensional**

El siguiente código muestra las primeras filas de la tabla maestra y calcula sus dimensiones totales.



In [ ]:
# 1. Visualización de la estructura
# Muestra las primeras 5 filas para verificar que las columnas de diferentes fuentes
# (ej. 'KPPS_1' junto a 'age' y 'UPDRS_Total') coexisten en la misma fila.
display(df_merged.head())

# 2. Auditoría de dimensiones
# .shape devuelve una tupla (Número de Filas, Número de Columnas)
# Esto es vital para detectar si el merge generó duplicados inesperados.
print(f"\nShape of the final merged DataFrame: {df_merged.shape}")

,Consecutivo,anio.eval_kpps,fecha.eval_kpps,accum.visits_kpps,visit.numb,reg.innn,KPSS1.F,KPSS1.S,KPSS1.TOT,KPSS.D1,...,PDQ8.index,PDQ39.mov.index,PDQ39.daily.index,PDQ39.Emotion.index,PDQ39.stigma.index,PDQ39.social.index,PDQ39.cogni.index,PDQ39.comm.index,PDQ39.discom.index,PDQ39.SI.index
0,NaN,NaN,NaN,NaN,1,41307,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,2,41307,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,1,70807,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2,70807,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,1,88927,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Shape of the final merged DataFrame: (4801, 311)


**Puntos Clave de la Revisión:**

**Integridad de Columnas:** Al ver el head(), deberías poder identificar variables de todos los dominios (demográficas, motoras, dolor y calidad de vida) en una sola vista.

**Control de Filas (shape[0]):** Compara el número de filas resultante con el número de filas de tu tabla base (df_cleaned).

Si el número aumentó drásticamente, es posible que haya habido un problema de duplicidad en las llaves (reg.innn, visit.numb) en alguna de las tablas secundarias (relación one-to-many no deseada).

Si usamos outer join, es normal que el número sea igual o mayor que la tabla más grande de las cuatro.

# **Depuración Final de la Cohorte Unificada**

Tras la fusión masiva de datos (outer join), es inevitable que existan filas con valores vacíos (por ejemplo, pacientes que tenían datos demográficos pero no contestaron la encuesta de dolor).

Para definir nuestra **Cohorte Analítica Final**, aplicamos un filtro de integridad: conservamos solo aquellos registros en la base de datos maestra que contengan respuestas completas en la escala KPPS.

**Filtrado por Integridad de la Variable Dependiente**

El siguiente script re-identifica las columnas de interés en el archivo fusionado y elimina los registros incompletos específicamente en la dimensión de dolor.

In [ ]:
# 1. Identificación de columnas KPPS en la matriz fusionada
# Buscamos nuevamente las variables de la escala de dolor dentro del 'df_merged'
kpss_columns_merged = df_merged.filter(like='KPSS').columns

# 2. Eliminación de filas sin datos de dolor
# Si un paciente tiene datos demográficos pero no tiene KPPS (NaN), se elimina del análisis.
df_merged_cleaned_kpss = df_merged.dropna(subset=kpss_columns_merged)

# 3. Verificación de la Cohorte Analítica
# Visualizamos la cabecera de la tabla definitiva
display(df_merged_cleaned_kpss.head())

# 4. Dimensiones finales
# Este '.shape' nos da el 'N' final real para los modelos estadísticos
print(f"\nShape of the cleaned merged DataFrame: {df_merged_cleaned_kpss.shape}")

,Consecutivo,anio.eval_kpps,fecha.eval_kpps,accum.visits_kpps,visit.numb,reg.innn,KPSS1.F,KPSS1.S,KPSS1.TOT,KPSS.D1,...,PDQ8.index,PDQ39.mov.index,PDQ39.daily.index,PDQ39.Emotion.index,PDQ39.stigma.index,PDQ39.social.index,PDQ39.cogni.index,PDQ39.comm.index,PDQ39.discom.index,PDQ39.SI.index
7,4170.0,2022.0,5/17/2022,1.0,1,100000,0.0,0.0,0.0,0.0,...,34.375,42.5,25.000000,33.333333,0.00,0.0,12.50,25.0,33.333333,21.458333
232,4475.0,2022.0,4/27/2022,1.0,1,100208,1.0,1.0,1.0,1.0,...,15.625,10.0,25.000000,12.500000,0.00,0.0,25.00,25.0,8.333333,13.229167
261,3509.0,2020.0,2020-06-02 00:00:00,1.0,1,100235,0.0,0.0,0.0,0.0,...,9.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
319,4537.0,2023.0,8/14/2023,10.0,9,104248,1.0,1.0,1.0,1.0,...,31.250,37.5,29.166667,41.666667,31.25,50.0,31.25,50.0,33.333333,38.020833
320,4538.0,2024.0,2024-02-12 00:00:00,10.0,10,104248,1.0,0.0,0.0,0.0,...,25.000,80.0,62.500000,8.333333,18.75,0.0,6.25,0.0,0.000000,21.979167



Shape of the cleaned merged DataFrame: (1067, 311)


**Interpretación del Flujo de Datos:**

**Antes del Merge:** Teníamos df_cleaned (solo datos KPPS limpios).

**Durante el Merge:** Añadimos df_sociodemograph, UPDRS, etc. usando outer join. Esto pudo haber traído pacientes "extra" que tenían datos demográficos pero NO tenían datos de dolor (sus columnas KPPS se llenaron con NaN).

**Después de este bloque** (df_merged_cleaned_kpss): Eliminamos a esos pacientes "extra". El resultado es una tabla que garantiza que cada fila tiene datos de dolor válidos Y, además, tiene adjunta toda la información demográfica disponible.

**Nota:** El número de filas aquí (df_merged_cleaned_kpss.shape[0]) debería ser muy similar o idéntico al de tu limpieza inicial (df_cleaned), pero ahora la tabla es mucho más ancha (más columnas).

# **Auditoría de Calidad de Datos: Porcentaje de Valores Faltantes**

Una vez consolidada la cohorte analítica (pacientes con datos de dolor válidos), debemos inspeccionar la integridad de las variables secundarias que utilizaremos como covariables o outcomes.

Este bloque calcula la proporción de datos perdidos en tres métricas clínicas clave: el índice de calidad de vida (PDQ-39 SI y PDQ-8) y el puntaje motor total (MDS-UPDRS).

**Cálculo de Porcentajes de Ausencia**

El siguiente script selecciona las columnas críticas y calcula qué porcentaje de filas contienen valores nulos (NaN).


In [ ]:
# 1. Cálculo de métricas de ausencia
# Seleccionamos las columnas específicas de interés clínico
cols_to_check = ['PDQ39.SI.index', 'PDQ8.index', 'UPDRS.TOTAL']

# Aplicamos la lógica:
# .isnull() -> Convierte a True (1) si falta dato, False (0) si existe.
# .mean() -> Calcula el promedio de 1s y 0s (esto nos da la proporción de faltantes).
# * 100 -> Convierte la proporción a porcentaje legible.
missing_percentages = df_merged_cleaned_kpss[cols_to_check].isnull().mean() * 100

# 2. Visualización de resultados
print("Percentage of missing values:")
print(missing_percentages)

Percentage of missing values:
PDQ39.SI.index    38.800375
PDQ8.index         3.467666
UPDRS.TOTAL        0.374883
dtype: float64


**Interpretación Clínica del Resultado:**

**< 5% de faltantes:** Generalmente se considera aceptable y manejable; a menudo se ignoran o se imputan de forma simple.

**5% - 20% de faltantes:** Requiere precaución. Si eliminas estos pacientes, podrías perder potencia estadística considerable. Podría ser necesario usar técnicas de imputación (ej. media, KNN o imputación múltiple).

**> 20% de faltantes:** Indica un problema sistemático en la recolección de datos para esa variable específica. Usar esta variable podría sesgar los resultados del estudio.

**Nota Técnica:** Este cálculo se realiza sobre df_merged_cleaned_kpss. Esto significa que estamos respondiendo a la pregunta: "De los pacientes que SÍ tienen evaluación de dolor, ¿cuántos de ellos NO tienen evaluación motora o de calidad de vida?"

# **Exportación y Persistencia del Dataset Final**

Una vez completada la fusión de las múltiples fuentes y depurada la cohorte analítica (eliminando registros sin datos de dolor), el último paso es guardar esta Base de Datos Maestra en el almacenamiento permanente (Google Drive).

Guardado en formato Excel
El siguiente código define la ruta de destino y exporta el DataFrame df_merged_cleaned_kpss como un archivo .xlsx.

In [ ]:
# 1. Definición de la ruta de salida
# Es recomendable usar un nombre descriptivo (ej. 'cleaned_kpss') para distinguirlo de los datos crudos.
output_path = '/content/drive/MyDrive/LCEN/08_Bases de Datos y Herramientas/08.2_ReMePARK/08.2.6_Unified ReMePARK 2024/Remepark_cleaned_kpss.xlsx'

# 2. Exportación del DataFrame
# Utilizamos 'to_excel' para generar un archivo compatible con Excel, SPSS o R.
df_merged_cleaned_kpss.to_excel(output_path, index=False)

# 3. Confirmación de éxito
print(f"Cleaned dataset saved to: {output_path}")

Cleaned dataset saved to: /content/drive/MyDrive/LCEN/08_Bases de Datos y Herramientas/08.2_ReMePARK/08.2.6_Unified ReMePARK 2024/Remepark_cleaned_kpss.xlsx


**Detalles Técnicos Importantes:**

**Formato .xlsx:** A diferencia del .csv, el formato Excel conserva mejor ciertos tipos de datos y es más fácil de compartir con colaboradores clínicos que no usan Python.

**Parámetro index=False:**

Al igual que hicimos en pasos anteriores, esta instrucción es vital.

Evita que Pandas guarde el índice numérico (0, 1, 2...) como una columna adicional A en el Excel.

Esto garantiza que, si volvemos a cargar este archivo en el futuro, no tendremos columnas "basura" (como Unnamed: 0).